In [23]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt

import glob
import random
import tensorflow as tf
from shutil import rmtree
from keras.models import load_model
from os.path import join, getctime, basename
from models import *
from utils import *
from data_preprocess import load_prep_data

%load_ext autoreload
%autoreload 2

aug_data = load_prep_data(time=500, debug=True, onehot=True)
x_train = aug_data['x_train']
y_train = aug_data['y_train']
x_valid = aug_data['x_valid']
y_valid = aug_data['y_valid']
x_test = aug_data['x_test']
y_test = aug_data['y_test']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Shape of x_train set: (6960, 22, 250)
Shape of y_train labels: (6960,)
Shape of x_valid set: (1500, 22, 250)
Shape of y_valid labels: (1500,)
Shape of X_test_prep set: (1772, 22, 250)
Shape of y_test_prep labels: (1772,)
Shape of y_train labels after categorical conversion: (6960, 4)
Shape of y_valid labels after categorical conversion: (1500, 4)
Shape of y_test labels after categorical conversion: (1772, 4)


In [24]:
input_shape = x_train.shape
x_train = tf.transpose( tf.expand_dims(x_train, axis=-1), perm=[0, 2, 3, 1])
x_valid = tf.transpose( tf.expand_dims(x_valid, axis=-1), perm=[0, 2, 3, 1])

print("x_train.shape: {}", format(x_train.shape))

x_train.shape: {} (6960, 250, 1, 22)


## 2 Layers CNN + GRU

In [ ]:
config = {
    # Network
    'num_inputs': x_train.shape[0],
    'input_shape': (input_shape[2], 1, input_shape[1]),
    'epochs': 50,
    'dropout': 0.5,
    'batch_size': 640
}
CNN2LayerGRU = CNN2LayerGRU()
CNN2LayerGRU.build_model(config)
history = CNN2LayerGRU.train(x_train, y_train, x_valid, y_valid, config, get_workpath('CNN2LayerGRU'))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 250, 1, 25)        13775     
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 84, 1, 25)         0         
 ng2D)                                                           
                                                                 
 batch_normalization_16 (Ba  (None, 84, 1, 25)         100       
 tchNormalization)                                               
                                                                 
 dropout_20 (Dropout)        (None, 84, 1, 25)         0         
                                                                 
 conv2d_17 (Conv2D)          (None, 84, 1, 50)         31300     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 28, 1, 50)        

In [ ]:
x_test_transposed = tf.transpose( tf.expand_dims(x_test, axis=-1), perm=[0, 2, 3, 1])
raw = CNN2LayerGRU.evaluate(x_test_transposed, y_test)
print("Raw Acc result: {}".format(raw[1]))
replace_model_if_better('CNN2LayerGRU', np.mean(raw[1]), CNN2LayerGRU, config)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

## 4 Layers CNN + GRU

In [ ]:
config = {
    # Network
    'num_inputs': x_train.shape[0],
    'input_shape': (input_shape[2], 1, input_shape[1]),
    'epochs': 50,
    'dropout': 0.5,
    'batch_size': 640
}
CNN4LayerGRU = CNN4LayerGRU()
CNN4LayerGRU.build_model(config)
history = CNN4LayerGRU.train(x_train, y_train, x_valid, y_valid, config, get_workpath('CNN4LayerGRU'))

In [ ]:
x_test_transposed = tf.transpose( tf.expand_dims(x_test, axis=-1), perm=[0, 2, 3, 1])
raw = CNN4LayerGRU.evaluate(x_test_transposed, y_test)
print("Raw Acc result: {}".format(raw[1]))
replace_model_if_better('CNN4LayerGRU', np.mean(raw[1]), CNN4LayerGRU, config)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()